### Imports, constants and settings

In [59]:
from typing import Tuple, Optional, Any
import hashlib
import binascii

In [60]:

# Set DEBUG to True to get a detailed debug output including
# intermediate values during key generation, signing, and
# verification. This is implemented via calls to the
# debug_print_vars() function.
#
# If you want to print values on an individual basis, use
# the pretty() function, e.g., print(pretty(foo)).
DEBUG = True

In [61]:

p = 0xFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFEFFFFFC2F
n = 0xFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFEBAAEDCE6AF48A03BBFD25E8CD0364141

# Points are tuples of X and Y coordinates and the point at infinity is
# represented by the None keyword.
G = (0x79BE667EF9DCBBAC55A06295CE870B07029BFCDB2DCE28D959F2815B16F81798, 0x483ADA7726A3C4655DA4FBFC0E1108A8FD17B448A68554199C47D08FFB10D4B8)


In [62]:
p

115792089237316195423570985008687907853269984665640564039457584007908834671663

In [63]:
n

115792089237316195423570985008687907852837564279074904382605163141518161494337

In [64]:
G

(55066263022277343669578718895168534326250603453777594175500187360389116729240,
 32670510020758816978083085130507043184471273380659243275938904335757337482424)

In [65]:
Point = Tuple[int, int]
Point

typing.Tuple[int, int]

In [66]:
# This implementation can be sped up by storing the midstate after hashing
# tag_hash instead of rehashing it all the time.
def tagged_hash(tag: str, msg: bytes) -> bytes:
    tag_hash = hashlib.sha256(tag.encode()).digest()
    return hashlib.sha256(tag_hash + tag_hash + msg).digest()


In [67]:
tag = "example_tag"
message = b"This is an example message"

tagged_hash_value = tagged_hash(tag, message)
tagged_hash_value.hex()

'379e0663cf310fb6b228f429f190aa1fe3c0b4ac37fe603a8b1a30fa025e9d34'

In [68]:
def is_infinite(P: Optional[Point]) -> bool:
    return P is None

In [69]:
P1: Optional[Point] = (1, 2)
P2: Optional[Point] = None

print(is_infinite(P1)) # Output: False
print(is_infinite(P2)) # Output: True

False
True


In [70]:
def x(P: Point) -> int:
    assert not is_infinite(P)
    return P[0]

# if is_infinite, returns assertionerror
# else returns P[0]

In [71]:
def y(P: Point) -> int:
    assert not is_infinite(P)
    return P[1]

# if is_infinite, returns assertionerror
# else returns P[1]

In [72]:
# point addition function
def point_add(P1: Optional[Point], P2: Optional[Point]) -> Optional[Point]:
    if P1 is None:
        return P2
    # returns p2 if p1 is none
    if P2 is None:
        return P1
    # returns p1 if p2 is none
    if (x(P1) == x(P2)) and (y(P1) != y(P2)):
        return None
    # if x(p1) and x(p2) are equal, and y(p1) and y(p2) arent equal, return none 
    # This corresponds to a situation where the two points are vertically aligned and cannot be added together.
    if P1 == P2:
        lam = (3 * x(P1) * x(P1) * pow(2 * y(P1), p - 2, p)) % p
    else:
        lam = ((y(P2) - y(P1)) * pow(x(P2) - x(P1), p - 2, p)) % p
    x3 = (lam * lam - x(P1) - x(P2)) % p
    return (x3, (lam * (x(P1) - x3) - y(P1)) % p)

In [73]:
# multiply a point by an integer, uses the point_add function
def point_mul(P: Optional[Point], n: int) -> Optional[Point]:
    R = None
    for i in range(256):
        if (n >> i) & 1:
            R = point_add(R, P)
        P = point_add(P, P)
    return R

In [74]:
# Returns bytes from an integer, 32 byte form
def bytes_from_int(x: int) -> bytes:
    return x.to_bytes(32, byteorder="big")

In [75]:
heloo = bytes_from_int(8327589273957892)
print("Length is: ", len(heloo))
int(heloo.hex(), 16)

Length is:  32


8327589273957892

In [76]:
# Returns bytes from the x coordinate of the Point P
def bytes_from_point(P: Point) -> bytes:
    return bytes_from_int(x(P))

In [77]:
test1 = bytes_from_point(G)
len(test1)
# So bytes_from_point returns a 32 byte string

32

In [78]:
# Takes two byte objects and returns the byte object that is the object obtained by XORing their corresponding bytes
def xor_bytes(b0: bytes, b1: bytes) -> bytes:
    return bytes(x ^ y for (x, y) in zip(b0, b1))

In [79]:
# Computes the y coordinate of an object given x, and returns the Point (x, y)
def lift_x(x: int) -> Optional[Point]:
    if x >= p:
        return None
    y_sq = (pow(x, 3, p) + 7) % p
    y = pow(y_sq, (p + 1) // 4, p)
    if pow(y, 2, p) != y_sq:
        return None
    return (x, y if y & 1 == 0 else p-y)

In [80]:
test_x = 0x79BE667EF9DCBBAC55A06295CE870B07029BFCDB2DCE28D959F2815B16F81798 # x coordinate of G
point = lift_x(test_x)
print(hex(point[1]))

0x483ada7726a3c4655da4fbfc0e1108a8fd17b448a68554199c47d08ffb10d4b8


In [81]:
# Generates int from bytes
def int_from_bytes(b: bytes) -> int:
    return int.from_bytes(b, byteorder="big")

In [82]:
hello = b'\x00\x00\x00\x01'
test_final = int_from_bytes(hello)
test_final

1